In [ ]:
import weaviate
import os
from weaviate import classes as wvc
client = weaviate.connect_to_local(
    headers={
        "x-openai-api-key": "chave-api-key"
    }
)

In [5]:
! docker compose exec -ti ollama ollama pull nomic-embed-text:latest
! docker compose exec -ti ollama ollama pull deepseek-r1:1.5b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏ 132 KB/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏ 1.1 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...   1% ▕                ▏ 1.5 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...   1% ▕                ▏ 2.8 MB/274 MB              

In [8]:
from weaviate import classes as wvc
client.collections.delete("Index")
client.collections.create(
    "Index",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.config.Configure.Generative.ollama(
        model="deepseek-r1:1.5b",
        api_endpoint="http://ollama:11434",
    )
)

In [9]:
# read csv file
import pandas as pd
from weaviate.util import generate_uuid5
collection = client.collections.get("Index")
df = pd.read_csv("trabalhos_60700009.csv")
df.columns = map(str.lower, df.columns)
# read only 10 objects
#df = df.head(5000)
with collection.batch.dynamic() as batch:
    for index, row in df.iterrows():
        batch.add_object(
            dict(row),
            uuid=generate_uuid5(row["indice"]),
        )
if collection.batch.failed_objects:
    print(collection.batch.failed_objects)
else:
    print("Imported with success")

Imported with success


In [10]:
collection.aggregate.over_all()

AggregateReturn(properties={}, total_count=4785)

In [11]:
collection.config.update(
    #generative_config=wvc.config.Configure.Generative.cohere()  # Update the generative module
    generative_config=wvc.config.Configure.Generative.openai()  # Update the generative module
)

In [66]:
query = collection.generate.near_text(
    #query="Quais os programas de pós-graduação e as instituições de ensino que mais se relacionam com a Organização do conhecimento de acordo com o Resumo e o Nome da Produção?",
    #query="Qual grau acadêmico tem trabalhado mais o tema Organização do conhecimento de acordo com o Resumo e o Nome da Produção?",
    #query="Qual é a região do páis onde temos a maior concentração e trabalhos sobre web semântica de acordo com o Resumo e o Nome da Produção?",
    #query="Quais são as ferramentas mais utilizadas em trabalos sobre ontologia nos Resumos e o Nome da Produção?",
    query="Qual a instituição tem mais trabalhos sobre Tesauro de acordo com o Resumo e o Nome da Produção?",
    grouped_task="Explique em um paragrafo qual a instituição tem mais trabalhos sobre Tesauro de acordo com o Resumo e o Nome da Produção, sem demonstrar o titulo ou resumos dos trabalhos?",
    limit=12,
    #auto_limit=5
)
for o in query.objects:
    print("###"*100 + str(o.metadata.distance))
    print("RESUMO: ", o.properties.get("ds_resumo"))
    print("NOME PRODUCAO: ", o.properties.get("nm_producao"))
    print("NOME PROGRAMA: ", o.properties.get("nm_programa"))
    print("NOME DA INSTITUIÇÃO: ", o.properties.get("nm_entidade_ensino"))
    print("NM_GRAU_ACADEMICO: ", o.properties.get("nm_grau_academico"))
    print("NM_REGIAO: ", o.properties.get("nm_regiao"))
    print("SG_UF_IES: ", o.properties.get("sg_uf_ies"))


# print("------"* 10)
# print(query.generated)


############################################################################################################################################################################################################################################################################################################None
RESUMO:  O acelerado desenvolvimento das tecnologias de informação e comunicação promove um avanço vertiginoso na veiculação do conhecimento. Tal fato incide diretamente na comunicação entre especialistas que, por meio de linguagens de especialidade, laçam mão de terminologias próprias e específicas para transmitirem conteúdos informacionais das mais diversas áreas científicas e tecnológicas. Linguagens Documentárias são modelos de representação do conhecimento que, servindo como instrumentos de controle terminológico, auxiliam o processo de indexação e recuperação de documentos por assunto. O tesauro é uma linguagem documentária caracterizada pela especificidade e pela complexidade exi

In [67]:
print(query.generated)

A instituição que tem mais trabalhos sobre Tesauro, de acordo com o Resumo e o Nome da Produção, é a Universidade Estadual Paulista Júlio de Mesquita Filho (UNESP). A instituição possui diversos trabalhos que abordam diferentes aspectos do Tesauro, como a construção de tesauros, a integração da Terminografia para a construção de tesauros, a análise do Tesauro UNESP sob a perspectiva da abordagem facetada, entre outros. Portanto, a UNESP se destaca como uma instituição com uma produção significativa na área de Tesauro.


In [56]:
len(query.objects)

12

In [ ]:
import weaviate
import os
from weaviate import classes as wvc
client = weaviate.connect_to_local(
    headers={
        "x-openai-api-key": "CHAVE_OPENAI"
    }
)

from weaviate import classes as wvc
client.collections.delete("Index")
client.collections.create(
    "Index",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.config.Configure.Generative.ollama(
        model="deepseek-r1:1.5b",
        api_endpoint="http://ollama:11434",
    )
)

import pandas as pd
from weaviate.util import generate_uuid5
collection = client.collections.get("Index")
df = pd.read_csv("trabalhos_60700009.csv")
df.columns = map(str.lower, df.columns)
with collection.batch.dynamic() as batch:
    for index, row in df.iterrows():
        batch.add_object(
            dict(row),
            uuid=generate_uuid5(row["indice"]),
        )
if collection.batch.failed_objects:
    print(collection.batch.failed_objects)
else:
    print("Imported with success")


collection.aggregate.over_all()

collection.config.update(
    generative_config=wvc.config.Configure.Generative.openai()  # Update the generative module
)

query = collection.generate.near_text(
    query="Qual a instituição tem mais trabalhos sobre Tesauro de acordo com o Resumo e o Nome da Produção?",
    grouped_task="Explique em um paragrafo qual a instituição tem mais trabalhos sobre Tesauro de acordo com o Resumo e o Nome da Produção",
    limit=12,
)
for o in query.objects:
    print("###"*100 + str(o.metadata.distance))
    print("RESUMO: ", o.properties.get("ds_resumo"))
    print("NOME PRODUCAO: ", o.properties.get("nm_producao"))
    print("NOME PROGRAMA: ", o.properties.get("nm_programa"))
    print("NOME DA INSTITUIÇÃO: ", o.properties.get("nm_entidade_ensino"))
    print("NM_GRAU_ACADEMICO: ", o.properties.get("nm_grau_academico"))
    print("NM_REGIAO: ", o.properties.get("nm_regiao"))
    print("SG_UF_IES: ", o.properties.get("sg_uf_ies"))

print(query.generated)